In [1]:
import numpy as np
import pandas as pd

In [2]:
def oneHot(Y):
    one_hot_Y = np.zeros((Y.size, Y.max() + 1))
    one_hot_Y[np.arange(Y.size), Y] = 1
    one_hot_Y = one_hot_Y.T
    return one_hot_Y

In [3]:
def getData():
    '''
    MNIST data set 
    x has 784 feature
    y is op value from 0 to 9 
    '''
    data = np.array(pd.read_csv('data/MNIST/MNIST_train.csv'))
    x = (data[:,1:]/255).T
    y = oneHot(data[:,0])
    return x,y
    

In [4]:
class Activation:
    @staticmethod
    def ReLU(Z, isDerivation=False):
        if isDerivation:
            return Z > 0
        return np.maximum(Z, 0)

    @staticmethod
    def sigmoid(Z, isDerivation=False):
        if isDerivation:
            op = Activation.sigmoid(Z)
            return op * (1- op)
        return 1/(1 + np.exp(-Z))

    @staticmethod
    def softmax(Z,isDerivation=False):
        if isDerivation:
            return 1
        Z = Z - np.max(Z, axis=0)
        A = np.exp(Z) / sum(np.exp(Z))
        return A

In [5]:
class Accuracy:
    @staticmethod
    def multiClass(target, prediction):
        return np.argmax(target, axis=0) == np.argmax(prediction, axis=0)

In [6]:
class Error:
    @staticmethod
    def meanSquareError(target, predicted, isDerivation=False):
        if isDerivation:
            return 2 * (predicted - target) / np.size(target)
        loss = np.power(target - predicted, 2)
        accuracy = Accuracy.multiClass(target, predicted)
        return {'loss': loss, 'accuracy': accuracy}
    
    @staticmethod
    def crossEntropyLoss(target, predicted,  isDerivation=False):
        if isDerivation:
            return predicted - target
        loss = -target * np.log(predicted + 10 ** -100)
        accuracy = Accuracy.multiClass(target,predicted)
        return {'loss': loss, 'accuracy': accuracy}
    
    @staticmethod
    def hiddenError(target, predicted):
        return target.T.dot(predicted)

In [7]:
class DenseLayer:
    op = []
    bias = []
    weight = []
    output = []
    isInput=False
    noOfNodes = 0
    activation = None
    def __init__(self, inputSize=0, outputSize=0, activation=None,isInput=False,input=[]):
        '''
        inputSize -> no.of.input feature 
        outputSize -> no.of.output
        '''
        if isInput:
            self.output = input
            self.isInput = True
            self.noOfNodes = input.shape[0]
        else :
            self.noOfNodes = inputSize
            self.activation = activation
            self.weight = np.random.rand(inputSize, outputSize) - 0.5
            self.bias = np.random.rand(inputSize,1) - 0.5

    def generateWeight(self,*r):
        '''
        Receive input as set that define the set shape
        '''
        return np.random.randn(*r) - 0.5

In [8]:
class NeuralNetwork:
    input = []
    target = []
    layers = []
    history = {'loss': [], 'accuracy':[]}
    loss = None

    def __init__(self, input, target, loss):
        self.loss = loss
        self.input = input
        self.target = target
        self.layers.append(DenseLayer(None,None,None,isInput=True, input=self.input))

    def append(self, node, activationFunction):
        preNode = self.layers[len(self.layers) -1]
        self.layers.append(DenseLayer(inputSize=node,outputSize=preNode.noOfNodes, activation=activationFunction))

    def train(self, epoch=100, lr=0.01):
        for i in range(epoch):
            for j in range(len(self.layers)):
                if not self.layers[j].isInput:
                    self.layers[j].output = self.forWord(self.layers[j], self.layers[j-1].output)
                if j == len(self.layers)-1 :
                    loss = self.loss(self.target,self.layers[j].output)
                    self.history['loss'].append(np.mean(loss['loss']))
                    self.history['accuracy'].append(np.mean(loss['accuracy']))
                    # index = len(self.history['loss']) -1
                    print(f"Epoch: {i+1} Loss: {self.history['loss'][-1]} Accuracy: {self.history['accuracy'][-1]}")
                    # calculate loss
                    self.backPropogation(loss['loss'],lr)

    def forWord(self, layer, input):
        layer.op = np.dot(layer.weight, input) + layer.bias
        return layer.activation(layer.op)

    def backPropogation(self, loss,lr):
        layer_length = len(self.layers)
        for index, layer in enumerate(self.layers[::-1]):
            if not layer.isInput:
                pervious_node = self.layers[layer_length - index - 2]
                oldw = np.copy(layer.weight)
                layer.weight, layer.bias = self.gradient(
                    layer.weight, layer.bias, pervious_node.output, loss,lr)
                loss = Error.hiddenError(oldw, loss)

    def gradient(self, w, b, x, err, lr=0.01):
        w = w - (1/len(x[0]) * (err.dot(x.T))) * lr
        b = b - (lr * np.mean(err, axis=1).reshape(b.shape))
        return w,b        

In [9]:
x,y = getData()

In [10]:
model = None
model = NeuralNetwork(np.copy(x),np.copy(y),Error.crossEntropyLoss)

model.append(10, Activation.ReLU)
model.append(10,Activation.softmax)
model.train(100)

Epoch: 1 Loss: 0.30110137833617723 Accuracy: 0.13997619047619048
Epoch: 2 Loss: 0.30675692296282164 Accuracy: 0.14007142857142857
Epoch: 3 Loss: 0.31337219278343054 Accuracy: 0.13997619047619048
Epoch: 4 Loss: 0.321234277926191 Accuracy: 0.14061904761904762
Epoch: 5 Loss: 0.33071721975162965 Accuracy: 0.14188095238095239
Epoch: 6 Loss: 0.34234776966772645 Accuracy: 0.14335714285714285
Epoch: 7 Loss: 0.35690198711307564 Accuracy: 0.14516666666666667
Epoch: 8 Loss: 0.3755298469555364 Accuracy: 0.14852380952380953
Epoch: 9 Loss: 0.40002912687264597 Accuracy: 0.15178571428571427
Epoch: 10 Loss: 0.4333196710591719 Accuracy: 0.15542857142857142
Epoch: 11 Loss: 0.4804057098580758 Accuracy: 0.15814285714285714
Epoch: 12 Loss: 0.5503249032936673 Accuracy: 0.16026190476190477
Epoch: 13 Loss: 0.6607957078950193 Accuracy: 0.16030952380952382
Epoch: 14 Loss: 0.8506420752288226 Accuracy: 0.15766666666666668
Epoch: 15 Loss: 1.2212369631632314 Accuracy: 0.15045238095238095
Epoch: 16 Loss: 2.1207766895

In [11]:
model1 = NeuralNetwork(np.copy(x),np.copy(y),Error.meanSquareError)

model1.append(10, Activation.ReLU)
model1.append(10,Activation.softmax)
model1.train(10, 0.1)

Epoch: 1 Loss: 0.10460289562838226 Accuracy: 0.0670952380952381


ValueError: operands could not be broadcast together with shapes (10,10) (784,10) 

# Debug

In [23]:
hw = np.random.rand(10, 784) - 0.5
hb = np.random.rand(10, 1) - 0.5
ow = np.random.rand(10, 10) - 0.5
ob = np.random.rand(10, 1) - 0.5

In [24]:
def forWord(x,w,b):
    return np.dot(w, x) + b

def backWord(err, w,b, x,lr = 0.01):
    m = 1/len(x[0])
    w = w - (m * (err.dot(x.T))) * lr
    b = b - (m * np.sum(err)) * lr
    return w, b

In [25]:
for i in range(100):
    
    #forword
    #Hidden
    hid_op = forWord(x,hw,hb)
    hid_act = Activation.ReLU(hid_op)

    #op layer
    op = forWord(hid_act,ow,ob)
    y_pred = Activation.softmax(op)

    #backword
    #output error
    op_err = y_pred - y
    ow,ob = backWord(op_err, ow, ob, hid_act, 0.1)

    # #hidden error
    hid_err = Error.hiddenError(ow,op_err) * Activation.ReLU(hid_op,True)
    hw,hb = backWord(hid_err, hw, hb, x, 0.1)

    loss = Error.crossEntropyLoss(y, y_pred)
    print(f"Epoch : {i + 1}, Loss : {np.mean(loss['loss'])}, Accuracy : {np.mean(loss['accuracy'])}")

Epoch : 1, Loss : 0.32805799202054386, Accuracy : 0.09066666666666667
Epoch : 2, Loss : 0.28324447498472743, Accuracy : 0.10083333333333333
Epoch : 3, Loss : 0.2622400445646596, Accuracy : 0.11247619047619048
Epoch : 4, Loss : 0.24918654891156053, Accuracy : 0.12404761904761905
Epoch : 5, Loss : 0.2401400435963171, Accuracy : 0.1391904761904762
Epoch : 6, Loss : 0.23335120091442968, Accuracy : 0.15783333333333333
Epoch : 7, Loss : 0.22791174533321443, Accuracy : 0.17676190476190476
Epoch : 8, Loss : 0.22334438556315384, Accuracy : 0.19323809523809524
Epoch : 9, Loss : 0.2193753420969677, Accuracy : 0.20754761904761904
Epoch : 10, Loss : 0.21584302021745685, Accuracy : 0.22104761904761905
Epoch : 11, Loss : 0.2126365299857345, Accuracy : 0.23261904761904761
Epoch : 12, Loss : 0.20968105717229918, Accuracy : 0.24323809523809523
Epoch : 13, Loss : 0.20692126776478323, Accuracy : 0.25211904761904763
Epoch : 14, Loss : 0.20431540274162077, Accuracy : 0.2603809523809524
Epoch : 15, Loss : 0.